In [1]:
import TrainingDataExtractor as tde
from deep_dynet import support

In [1]:
from os import path
import sys
sys.path.append(path.abspath('./stanford_parser'))

from stanford_parser.parser import Parser

SyntaxError: Non-ASCII character '\xc6' in file stanford_parser/parser.py on line 132, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details (parser.py, line 132)

### Node relations prep

In [1]:
relationspath = 'resources/labels/relations.txt'
with open(relationspath) as f:
    relations = f.readlines()
    relations = map(lambda rel: rel.strip(), relations)
    relations_dict = {rel: index for rel, index in zip(relations, range(len(relations)))}

In [4]:
relations[0]

':ARG0'

In [5]:
relations

[':ARG0',
 ':ARG1',
 ':ARG2',
 ':ARG3',
 ':ARG4',
 ':ARG5',
 ':ARG6',
 ':accompanier',
 ':age',
 ':beneficiary',
 ':calendar',
 ':century',
 ':compared-to',
 ':concession',
 ':condition',
 ':conj-as-if',
 ':consist',
 ':day',
 ':dayperiod',
 ':decade',
 ':degree',
 ':destination',
 ':direction',
 ':domain',
 ':duration',
 ':era',
 ':example',
 ':extent',
 ':frequency',
 ':instrument',
 ':li',
 ':location',
 ':manner',
 ':medium',
 ':mod',
 ':mode',
 ':month',
 ':name',
 ':op',
 ':ord',
 ':part',
 ':path',
 ':polarity',
 ':polite',
 ':poss',
 ':prep-against',
 ':prep-along-with',
 ':prep-amid',
 ':prep-among',
 ':prep-as',
 ':prep-at',
 ':prep-by',
 ':prep-except',
 ':prep-following',
 ':prep-for',
 ':prep-from',
 ':prep-in',
 ':prep-in-addition-to',
 ':prep-instead',
 ':prep-into',
 ':prep-on',
 ':prep-on-behalf',
 ':prep-out',
 ':prep-through',
 ':prep-to',
 ':prep-toward',
 ':prep-under',
 ':prep-versus',
 ':prep-with',
 ':prep-within',
 ':prep-without',
 ':purpose',
 ':quant',
 ':qu

In [6]:
relations_dict[':ARG0']

0

In [7]:
len(relations)

98

In [8]:
# Dep parsing test
parser = Parser()

dependencies = parser.parseToStanfordDependencies('That is life .')

tupleResult = [(rel, gov.text, dep.text) for rel, gov, dep in dependencies.dependencies]
print tupleResult

[('nsubj', 'life', 'That'), ('cop', 'life', 'is')]


In [9]:
from os import listdir

def read_data(type, dataset=None):
    data = []
    mypath = 'resources/alignments/split/' + type
    print(mypath)
    if dataset is None:
        for f in listdir(mypath):
            mypath_f = mypath + "/" + f
            print(mypath_f)
            data += tde.generate_training_data(mypath_f, False)
    else:
        mypath_f = mypath + "/" + dataset
        print(mypath_f)
        data = tde.generate_training_data(mypath_f, verbose=False, withDependencies=True)
    return data

data = read_data('training', 'deft-p2-amr-r1-alignments-training-dfa.txt')

resources/alignments/split/training
resources/alignments/split/training/deft-p2-amr-r1-alignments-training-dfa.txt


100%|██████████| 6455/6455 [03:33<00:00, 30.23it/s]
CRITICAL:root:Failed: 5302 out of 6455


In [10]:
len(data)

1153

In [11]:
data[1000]

('That really sucks ! :(',
 ['SH_that', 'SH_really', 'SH_suck-03', 'RL_degree', 'RL_ARG0', 'DN', 'DN'],
 '(s / suck-03~e.2 \n      :ARG0 (t / that~e.0) \n      :degree (r / really~e.1))\n',
 {1: (2, 'advmod'), 2: (0, 'amod')})

In [12]:
len(data[0])

4

In [13]:
data[20][0]

"Either way , they 've got a big problem ."

In [14]:
data[20][1]

['SH_either',
 'SH_way',
 'RL_mod',
 'DN',
 'SH_they',
 'DN',
 'SH_get-02',
 'RL_ARG0',
 'RL_mod',
 'DN',
 'SH_big',
 'SH_problem',
 'RL_degree',
 'RR_ARG1',
 'DN']

In [15]:
vocab_acts = support.Vocab.from_list(['SH', 'RL', 'RR', 'DN', 'SW'])
action_sequence = support.oracle_actions_to_action_index(data[20][1], vocab_acts)

In [16]:
action_sequence

[action: SH label: either index: 0,
 action: SH label: way index: 0,
 action: RL label: mod index: 1,
 action: DN label: None index: 3,
 action: SH label: they index: 0,
 action: DN label: None index: 3,
 action: SH label: get-02 index: 0,
 action: RL label: ARG0 index: 1,
 action: RL label: mod index: 1,
 action: DN label: None index: 3,
 action: SH label: big index: 0,
 action: SH label: problem index: 0,
 action: RL label: degree index: 1,
 action: RR label: ARG1 index: 2,
 action: DN label: None index: 3]

In [17]:
data[20][2]

'(g / get-02~e.5 \n      :ARG0 (t / they~e.3) \n      :ARG1 (p / problem~e.8 \n            :degree (b / big~e.7)) \n      :mod (w / way~e.1 \n            :mod (e / either~e.0)))\n'

#### Obtain for each training example the following: The sentence, the sequence of action indices, the labels and the dependencies

In [18]:
sentences = [d[0] for d in data]

In [19]:
sentences

['company could make a fortune off of this guy .',
 'Did person apologize to person ?',
 'Mend some fences and get this country moving .',
 'He could call it the tour tour .',
 'I will stick around until the end .',
 'No one',
 'I think person drops out...',
 'Ok , my answer is no one and person wins the event .',
 'Nuts .',
 'person is not quite bright .',
 'The problem with this regime is aid is not distributed equally .',
 'Aid is now patronage , good party members get it .',
 'Thats what a Hardcore Libertarian would say .',
 'No surprise there .',
 'If he were out of the race , person might have had a chance .',
 'Where is the morality in creating a life when food and clean water are rare commodities ?',
 'Where is the morality when disease is spread ?',
 'Where is the morality when a women is the victim of domestic violence ?',
 'I am with you .',
 "Now am I just a genius who does n't know it , or is it really not THAT hard to understand such a simple concept ?",
 "Either way , th

In [20]:
amrs = [d[2] for d in data]

In [21]:
amrs

['(p / possible-01~e.1 \n      :ARG1 (m / make-05~e.2 \n            :ARG0 (c / company :wiki "Hallmark_Cards" \n                  :name (n / name :op1 "Hallmark"~e.0)) \n            :ARG1 (f / fortune~e.4 \n                  :source~e.6 (g / guy~e.8 \n                        :mod (t / this~e.7)))))\n',
 '(a / apologize-01~e.2 :mode~e.5 interrogative~e.5 \n      :ARG0 (p / person :wiki "Sarah_Palin" \n            :name (n / name :op1 "Palin"~e.1)) \n      :ARG2~e.3 (p2 / person :wiki "Gabrielle_Giffords" \n            :name (n2 / name :op1 "Giffords"~e.4)))\n',
 '(a / and~e.3 \n      :op1 (m / mend-01~e.0 \n            :ARG1 (f / fence~e.2 \n                  :quant (s / some~e.1))) \n      :op2 (g / get-04~e.4 \n            :ARG1 (m2 / move-03~e.7 \n                  :ARG1 (c / country~e.6 \n                        :mod (t / this~e.5)))))\n',
 '(p / possible-01~e.1 \n      :ARG1 (c / call-01~e.2 \n            :ARG0 (h / he~e.0) \n            :ARG1 (i / it~e.3) \n            :ARG2 (t / 

In [22]:
actions = [support.oracle_actions_to_action_index(d[1], vocab_acts) for d in data]

In [23]:
actions[0][0]

action: SH label: company index: 0

In [24]:
action_indices = [[a.index for a in actions_list] for actions_list in actions]

In [25]:
action_indices

[[0, 0, 0, 3, 0, 3, 3, 0, 0, 1, 2, 2, 3, 4, 1, 2],
 [3, 0, 0, 1, 3, 0, 2, 0, 2],
 [0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 1, 0, 1, 2, 2, 3],
 [0, 0, 0, 0, 2, 3, 0, 2, 3, 3, 4, 1, 2],
 [0, 3, 0, 1, 3, 0, 3, 0, 2, 2, 3],
 [0, 3],
 [0, 0, 1, 0, 0, 1, 2, 3],
 [3, 3, 0, 0, 1, 3, 0, 3, 0, 1, 0, 0, 1, 3, 0, 2, 2, 2, 3],
 [0, 3],
 [0, 3, 0, 0, 1, 0, 1, 1, 3],
 [3, 0, 3, 0, 0, 1, 2, 3, 0, 3, 0, 0, 0, 3, 4, 1, 2, 1, 2],
 [0, 3, 0, 0, 1, 1, 3, 0, 0, 0, 1, 1, 0, 1, 2, 3, 3],
 [3, 3, 3, 0, 0, 1, 3, 0, 1, 3],
 [0, 0, 1, 0, 2, 3],
 [3, 0, 3, 0, 1, 3, 3, 0, 2, 3, 0, 0, 3, 3, 3, 0, 3, 4, 1, 2, 1],
 [0,
  3,
  3,
  0,
  1,
  3,
  0,
  3,
  0,
  2,
  3,
  0,
  0,
  1,
  0,
  0,
  1,
  2,
  3,
  0,
  0,
  1,
  1,
  2,
  2,
  3],
 [3, 3, 3, 0, 0, 2, 0, 3, 0, 1, 2, 3],
 [3, 3, 3, 0, 0, 2, 3, 0, 3, 3, 0, 1, 3, 0, 0, 1, 2, 2, 3],
 [0, 3, 3, 0, 2, 3],
 [0,
  3,
  0,
  0,
  3,
  0,
  1,
  1,
  3,
  3,
  0,
  0,
  1,
  0,
  2,
  2,
  3,
  0,
  1,
  1,
  3,
  3,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  3,
  0,
  3,
  0,


In [26]:
action_labels = [[a.label for a in actions_list] for actions_list in actions]

In [27]:
action_labels

[['company',
  'possible-01',
  'make-05',
  None,
  'fortune',
  None,
  None,
  'this',
  'guy',
  'mod',
  'source',
  'ARG1',
  None,
  None,
  'ARG0',
  'ARG1'],
 [None,
  'person',
  'apologize-01',
  'ARG0',
  None,
  'person',
  'ARG2',
  'interrogative',
  'mode'],
 ['mend-01',
  'some',
  'fence',
  'quant',
  'ARG1',
  'and',
  'op1',
  'get-04',
  'this',
  'country',
  'mod',
  'move-03',
  'ARG1',
  'ARG1',
  'op2',
  None],
 ['he',
  'possible-01',
  'call-01',
  'it',
  'ARG1',
  None,
  'tour',
  'ARG2',
  None,
  None,
  None,
  'ARG0',
  'ARG1'],
 ['i',
  None,
  'stick-around-03',
  'ARG0',
  None,
  'until',
  None,
  'end-01',
  'op1',
  'time',
  None],
 ['no-one', None],
 ['i', 'think-01', 'ARG0', 'person', 'drop-out-04', 'ARG0', 'ARG1', None],
 [None,
  None,
  'i',
  'answer-01',
  'ARG0',
  None,
  'no-one',
  None,
  'and',
  'op1',
  'person',
  'win-01',
  'ARG0',
  None,
  'event',
  'ARG2',
  'op2',
  'ARG2',
  None],
 ['nuts', None],
 ['person', None, '

### Check all the reduce actions labels

In [28]:
relation_labels =  [[a.label for a in actions_list if a.action == "RL" or a.action == "RR"] for actions_list in actions]

In [29]:
import operator

relation_labels_all = reduce(operator.add, relation_labels)

In [30]:
relation_labels

[['mod', 'source', 'ARG1', 'ARG0', 'ARG1'],
 ['ARG0', 'ARG2', 'mode'],
 ['quant', 'ARG1', 'op1', 'mod', 'ARG1', 'ARG1', 'op2'],
 ['ARG1', 'ARG2', 'ARG0', 'ARG1'],
 ['ARG0', 'op1', 'time'],
 [],
 ['ARG0', 'ARG0', 'ARG1'],
 ['ARG0', 'op1', 'ARG0', 'ARG2', 'op2', 'ARG2'],
 [],
 ['polarity', 'mod', 'ARG1'],
 ['mod', 'topic', 'polarity', 'manner', 'ARG1', 'domain'],
 ['time', 'domain', 'part-of', 'mod', 'ARG0', 'ARG1-of'],
 ['mod', 'ARG0'],
 ['polarity', 'location'],
 ['ARG1', 'ARG2', 'ARG0', 'ARG1', 'condition'],
 ['location',
  'ARG1',
  'op1',
  'ARG1-of',
  'op2',
  'ARG1-of',
  'domain',
  'time',
  'domain'],
 ['location', 'ARG1', 'time'],
 ['location', 'ARG1', 'mod', 'ARG0', 'time'],
 ['accompanier'],
 ['mod',
  'domain',
  'polarity',
  'ARG1',
  'ARG0-of',
  'op1',
  'mod',
  'polarity',
  'degree',
  'mod',
  'degree',
  'ARG1-of',
  'ARG1',
  'ARG1',
  'op2'],
 ['mod', 'ARG0', 'mod', 'degree', 'ARG1'],
 ['mod', 'ARG0', 'ARG1'],
 ['mod', 'ARG0', 'quant', 'ARG0', 'ARG0', 'ARG1-of']

In [32]:
dataset_relation_labels = set(relation_labels_all)

In [33]:
dataset_relation_labels

{'ARG0',
 'ARG0-of',
 'ARG1',
 'ARG1-of',
 'ARG2',
 'ARG2-of',
 'ARG3',
 'ARG3-of',
 'ARG4',
 'accompanier',
 'beneficiary',
 'compared-to',
 'concession',
 'concession-of',
 'condition',
 'consist-of',
 'dayperiod',
 'degree',
 'direction',
 'domain',
 'duration',
 'example',
 'extent',
 'frequency',
 'instrument-of',
 'li',
 'location',
 'location-of',
 'manner',
 'manner-of',
 'medium',
 'mod',
 'mode',
 'op1',
 'op2',
 'op3',
 'op4',
 'op5',
 'ord-of',
 'part-of',
 'path',
 'polarity',
 'poss',
 'poss-of',
 'prep-against',
 'prep-along-with',
 'prep-amid',
 'prep-as',
 'prep-at',
 'prep-for',
 'prep-in',
 'prep-in-addition-to',
 'prep-on',
 'prep-to',
 'prep-under',
 'prep-with',
 'prep-without',
 'purpose',
 'quant',
 'quant-of',
 'source',
 'source-of',
 'subevent',
 'time',
 'topic',
 'topic-of'}

In [34]:
len(dataset_relation_labels)

66

In [35]:
for rel in dataset_relation_labels:
    rel = ":" + rel
    if rel not in relations_dict.keys():
        print 'Missing relation: %s' % rel

Missing relation: :op4
Missing relation: :op5
Missing relation: :op1
Missing relation: :op2
Missing relation: :op3
Missing relation: :source-of
Missing relation: :ARG3-of
Missing relation: :location-of
Missing relation: :quant-of
Missing relation: :ARG1-of
Missing relation: :manner-of
Missing relation: :ARG2-of
Missing relation: :consist-of
Missing relation: :part-of
Missing relation: :ARG0-of
Missing relation: :poss-of
Missing relation: :ord-of
Missing relation: :concession-of
Missing relation: :instrument-of
Missing relation: :topic-of


In [36]:
dependencies = [d[3] for d in data]

In [37]:
dependencies

[{0: (2, 'nsubj'),
  1: (2, 'aux'),
  3: (4, 'det'),
  4: (2, 'dobj'),
  7: (8, 'det'),
  8: (2, 'prep_off_of')},
 {0: (2, 'aux'), 1: (2, 'nsubj'), 4: (2, 'prep_to')},
 {1: (2, 'det'),
  2: (0, 'dobj'),
  4: (0, 'conj_and'),
  5: (6, 'det'),
  6: (4, 'dobj'),
  7: (6, 'partmod')},
 {0: (2, 'nsubj'),
  1: (2, 'aux'),
  3: (2, 'iobj'),
  4: (6, 'det'),
  5: (6, 'nn'),
  6: (2, 'dobj')},
 {0: (2, 'nsubj'),
  1: (2, 'aux'),
  3: (2, 'prt'),
  5: (6, 'det'),
  6: (2, 'prep_until')},
 {0: (1, 'det')},
 {0: (1, 'nsubj'), 2: (3, 'nn'), 3: (1, 'dobj'), 4: (1, 'prt')},
 {0: (6, 'advmod'),
  2: (3, 'poss'),
  3: (6, 'nsubj'),
  4: (6, 'cop'),
  5: (6, 'det'),
  8: (6, 'conj_and'),
  9: (6, 'rcmod'),
  10: (11, 'det'),
  11: (9, 'dobj')},
 {},
 {0: (4, 'nsubj'), 1: (4, 'cop'), 2: (4, 'neg'), 3: (4, 'advmod')},
 {0: (1, 'det'),
  1: (6, 'nsubj'),
  3: (4, 'det'),
  4: (1, 'prep_with'),
  5: (6, 'cop'),
  7: (9, 'auxpass'),
  8: (9, 'neg'),
  9: (6, 'rcmod'),
  10: (9, 'advmod')},
 {0: (3, 'nsubj'),

#### For the initial LSTM training we will have: Input = list of tokens, Output = list of actions. If the action is a reduce or swap, reinsert the same token at input at the next time step

#### Specifically, the LSTM will receive the word embedding of the first two tokens on the stack and the embedding of the current token on the buffer. The output will be an action index. The action index is analyzed and based on it we decide whether the next fed token is the same or the next one.

In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [39]:
tokenizer = Tokenizer(filters="", lower=True, split=" ")

In [40]:
tokenizer.fit_on_texts(sentences)

In [41]:
import numpy as np

In [42]:
sequences = np.asarray(tokenizer.texts_to_sequences(sentences))

In [43]:
sequences

array([[124, 78, 90, 5, 1389, 192, 10, 13, 356, 1],
       [99, 20, 1907, 12, 20, 3], [1859, 71, 1814, 21, 65, 13, 14, 265, 1],
       ..., [31, 7, 149, 1400, 63, 5, 291, 3],
       [8, 208, 34, 18, 4, 111, 151, 1462, 1], [8, 438, 15, 169, 1]], dtype=object)

In [44]:
word_index = tokenizer.word_index

In [45]:
word_index

{'suicidal': 764,
 'negs': 765,
 'hanging': 766,
 'hate': 441,
 'localized': 767,
 'forget': 768,
 'stinks': 442,
 'whose': 769,
 'feeding': 770,
 'vile': 1452,
 'attracted': 1215,
 'snowing': 772,
 'bureaucrats': 773,
 'voted': 443,
 'anticlimax': 774,
 'under': 775,
 'sorry': 260,
 'worth': 321,
 '@': 225,
 'rise': 444,
 'misunderstanding': 776,
 'every': 777,
 'bringing': 322,
 'school': 778,
 'basics': 779,
 'prize': 780,
 'companies': 445,
 'solution': 446,
 'poppy': 782,
 'ups': 783,
 'overdo': 784,
 'shut': 1912,
 'likely': 787,
 'kratom': 788,
 'implemented': 789,
 'it...': 790,
 'even': 149,
 'established': 791,
 'prisons': 792,
 'integrity': 793,
 '+': 794,
 'sexuality': 1702,
 'above': 795,
 'new': 193,
 'increasing': 797,
 'ever': 194,
 'told': 798,
 'hero': 800,
 'never': 232,
 'here': 60,
 'nutjob': 801,
 'reported': 449,
 'protection': 802,
 'aftermath': 803,
 '100': 804,
 'digits': 450,
 'kids': 493,
 'changed': 806,
 'vicepresident': 807,
 ':wink': 808,
 'military': 11

In [46]:
len(word_index)

2130

#### Shuffle data

In [47]:
indices = np.arange(sequences.shape[0])

In [48]:
np.random.shuffle(indices)

In [49]:
indices

array([ 775,  818,   59, ...,  796, 1094,  688])

In [50]:
sequences = sequences[indices]

In [51]:
actions = np.asarray(action_indices)[indices]

In [52]:
action_labels = np.asarray(action_labels)[indices]

In [53]:
dependencies = [dependencies[i] for i in indices]

In [54]:
amrs = np.asanyarray(amrs)[indices]

In [55]:
sequences

array([[73, 1546], [16, 33, 50, 130, 121, 281, 3], [2073, 1140], ...,
       [19, 25, 5, 1738, 3], [25, 30, 27, 5, 140, 129, 10, 7, 1],
       [79, 8, 664, 116, 60, 3]], dtype=object)

In [56]:
actions

array([[3, 0], [3, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0, 2], [0, 3], ...,
       [3, 0, 3, 0, 1, 0, 2], [0, 3, 0, 1, 3, 0, 0, 1, 3, 0, 2, 2, 3],
       [3, 0, 0, 1, 0, 2, 0, 2, 0, 2]], dtype=object)

In [57]:
action_labels

array([[None, 'brugmansia'],
       [None, 'we', 'really', 'want-01', 'mod', 'ARG0', 'another', 'superpower', 'mod', 'ARG1', 'interrogative', 'mode'],
       ['warm-07', None], ...,
       [None, 'they', None, 'endanger-01', 'ARG0', 'interrogative', 'mode'],
       ['they', None, 'have-03', 'ARG0', None, 'hard-02', 'go-01', 'ARG1-of', None, 'it', 'ARG1', 'ARG1', None],
       [None, 'i', 'miss-02', 'ARG0', 'something', 'ARG1', 'here', 'location', 'interrogative', 'mode']], dtype=object)

In [58]:
amrs

array(['(b / brugmansia~e.1)\n',
       '(w / want-01~e.3 :mode~e.6 interrogative~e.6 \n      :ARG0 (w2 / we~e.1) \n      :ARG1 (s / superpower~e.5 \n            :mod (a / another~e.4)) \n      :mod (r / really~e.2))\n',
       '(t / temperature :mode interrogative \n      :ARG1-of (w / warm-07~e.0 \n            :degree~e.0 (m / more~e.0)))\n',
       ...,
       '(e / endanger-01~e.3 :mode~e.4 interrogative~e.4 \n      :ARG0 (t / they~e.1))\n',
       '(h / have-03~e.2 \n      :ARG0 (t / they~e.0) \n      :ARG1 (g / go-01~e.5 \n            :ARG1~e.6 (i / it~e.7) \n            :ARG1-of (h2 / hard-02~e.4)))\n',
       '(m / miss-02~e.2 :mode~e.5 interrogative~e.5 \n      :ARG0 (i / i~e.1) \n      :ARG1 (s / something~e.3) \n      :location (h / here~e.4))\n'], 
      dtype='|S1013')

In [59]:
num_train_samples = int(0.95 * sequences.shape[0])

In [60]:
x_train = sequences[:num_train_samples]
y_train = actions[:num_train_samples]
y_train_labels = action_labels[:num_train_samples]
amrs_train = amrs[:num_train_samples]
dependencies_train = dependencies[:num_train_samples]

x_test = sequences[num_train_samples:]
y_test = actions[num_train_samples:]
y_test_labels = action_labels[num_train_samples:]
amrs_test = amrs[num_train_samples:]
dependencies_test = dependencies[num_train_samples:]

In [61]:
print x_train.shape
print y_train.shape
print y_train_labels.shape
print amrs_train.shape

(1095,)
(1095,)
(1095,)
(1095,)


In [62]:
print len(dependencies_train)

1095


#### Prepare the embedding layer

In [63]:
embeddings_index = {}
f = open('./resources/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [64]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 2, EMBEDDING_DIM))
not_found = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        not_found.append(word)

print 'Not found: {}'.format(not_found)

Not found: ['negs', 'it...', 'nutjob', ':wink', 'lable', 'broadcast-program', 'soon...', 'monetary-quantity', "don'cha", 'regretable', 'numpty', '25+', '",', '".', '"?', 'music-key', 'blog-01', 'temporal-quantity', 'david...', 'world-region', '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!', "'.", 'funny...', 'colenel', 'definate', '@-@', 'v=iaadvod2srq', 'temperatures.?', 'miniture', 'afelm', 'bday', '!!!!!!1', '^^^', 'though...', 'political-party', 'aeroplace', '#9', '#8', '#3', '#2', '#1', '#7', '#5', 'me......', '@/@', 'a....well', 'said..', 'out...', 'ehh', 'one....', 'country-region', '@-', '****.', '#4', '...where', 'educated(?)', '****', 'date-entity', 'yes..', 'hmm...good', 'indeciperable', 'regional-group', 'religious-group', '."', 'minutiea', 'not...it', 'ethnic-group', 'so....who', 'worship-place', '*******', 'jail...', 'dosent', 'typical...', 'distance-quantity', 'about.?', "won't", 'anyhoo', 'warmist', 'aircraft-type', '#10', '...and', 'slowy', 'tbh', '=/', 'sports-facilit

#### Prepare the proper data set:
#### Input: Buffer top, First three elements on the stack, previous action index, stack[0] deps on stack[1], stack[1] deps on stack[0], stack[0] deps on buffer[0], buffer[0] deps on stack[1], stack[0] deps on stack[2], stack[2] deps on stack[0].
#### If the current action is shift, the next action will have the next token in the buffer and updated stack elements.
#### Else, the same element on the buffer is fed and the elements from the stack are updated
#### Do not consider instances with more than 30 actions for the moment.

In [65]:
import sklearn.preprocessing
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(5))

SH = 0
RL = 1
RR = 2
DN = 3
SW = 4
NONE = 5

NO_RELATION_LABEL = len(relations)

max_len = 30
no_word_index = (len(word_index)) + 1

def generate_dataset(x, y, y_labels, dependencies):
    x_full = np.zeros((len(x), max_len, 15), dtype=np.int32)
    y_full = np.full((len(y), max_len), dtype=np.int32, fill_value=NONE)
    y_full_relation_labels = np.full((len(y), max_len), dtype=np.int32, fill_value=NO_RELATION_LABEL)
    
    lengths = []
    filtered_count = 0

    for action_sequence, action_labels, tokens_sequence, deps, i in zip(y, y_labels, x, dependencies, range(len(y))):
        next_action_token = tokens_sequence[0]
        next_action_stack = [no_word_index, no_word_index, no_word_index, no_word_index] 
        next_action_prev_action = NONE
        tokens_sequence_index = 0
        
        features_matrix = []
        relation_labels = []
        
        lengths.append(len(action_sequence))
        if len(action_sequence) > 30:
            filtered_count += 1
            continue
        for action, j in zip(action_sequence, range(len(action_sequence))):
            if next_action_prev_action != NONE:
                next_action_prev_action_ohe = label_binarizer.transform([next_action_prev_action])[0, :]
            else:
                next_action_prev_action_ohe = [0, 0, 0, 0, 0]
            
            dep_0_on_1 = 0
            dep_1_on_0 = 0
            dep_0_on_2 = 0
            dep_2_on_0 = 0
            dep_0_on_b = 0
            dep_b_on_0 = 0
            
            if next_action_stack[0] in  deps.keys() and deps[next_action_stack[0]][0] == next_action_stack[1]:
                dep_0_on_1 = 1
            if next_action_stack[1] in  deps.keys() and deps[next_action_stack[1]][0] == next_action_stack[0]:
                dep_1_on_0 = 1
            if next_action_stack[0] in  deps.keys() and deps[next_action_stack[0]][0] == next_action_stack[2]:
                dep_0_on_2 = 1
            if next_action_stack[2] in  deps.keys() and deps[next_action_stack[2]][0] == next_action_stack[0]:
                dep_2_on_0 = 1
            if next_action_stack[0] in  deps.keys() and deps[next_action_stack[0]][0] == next_action_token:
                dep_0_on_b = 1
            if next_action_token in  deps.keys() and deps[next_action_token][0] == next_action_stack[0]:
                dep_b_on_0 = 1
            # next_action_prev_action_ohe = []
            features = np.concatenate((np.asarray([next_action_token, next_action_stack[0], 
                                             next_action_stack[1], next_action_stack[2]]),
                        next_action_prev_action_ohe,
                                      np.asarray([dep_0_on_1, dep_1_on_0, dep_0_on_2, dep_2_on_0, dep_0_on_b, dep_b_on_0])))
            if action == SH:
                tokens_sequence_index += 1
                next_action_stack = [next_action_token] + next_action_stack
                if tokens_sequence_index < len(tokens_sequence):
                    next_action_token = tokens_sequence[tokens_sequence_index]
                else:
                    next_action_token = no_word_index 
            if action == RL:
                next_action_stack = [next_action_stack[0]] + next_action_stack[2:]
            if action == RR:
                next_action_stack = [next_action_stack[1]] + next_action_stack[2:]
            if action == DN:
                tokens_sequence_index += 1
                if tokens_sequence_index < len(tokens_sequence):
                    next_action_token = tokens_sequence[tokens_sequence_index]
                else:
                    next_action_token = no_word_index
            if action == SW:
                next_action_stack = [next_action_stack[0], next_action_stack[2], next_action_stack[1]] + next_action_stack[3:]
            next_action_prev_action = action
            features_matrix.append(features)
            
            if action_labels[j] is not None:
                proper_action_label = ":" + action_labels[j]
            else:
                proper_action_label = ""
            if (action == RL or action == RR) and proper_action_label in relations_dict.keys(): 
                relation_labels.append(relations_dict[proper_action_label])
            else:
                relation_labels.append(NO_RELATION_LABEL)
                       
        if tokens_sequence_index != len(tokens_sequence):
            raise Exception("There was a problem at training instance " + str(i) + "\n")

        features_matrix = np.concatenate((np.asarray(features_matrix),
                                          np.zeros((max_len - len(features_matrix), 15), dtype=np.int32)))
        actions = np.concatenate((np.asarray(action_sequence),
                                np.full((max_len - len(action_sequence)), dtype=np.int32, fill_value=NONE)))
        relation_labels =  np.concatenate((np.asarray(relation_labels),
                                np.full((max_len - len(relation_labels)), dtype=np.int32, fill_value=NO_RELATION_LABEL)))
        x_full[i, :, :] = features_matrix
        y_full[i, :] = actions
        y_full_relation_labels[i, :] = relation_labels
    return x_full, y_full, y_full_relation_labels, lengths, filtered_count

(x_train_full, y_train_full, y_train_relation_labels_full, lengths_train, filtered_count_tr) = generate_dataset(x_train, y_train, y_train_labels, dependencies_train)
(x_test_full, y_test_full, y_test_relation_labels_full, lengths_test, filtered_count_test) = generate_dataset(x_test, y_test, y_test_labels, dependencies_test)

In [66]:
np.asarray(lengths_train).mean()

12.639269406392694

In [67]:
filtered_count_tr

30

In [68]:
x_train_full.shape

(1095, 30, 15)

In [69]:
y_train_full.shape

(1095, 30)

In [70]:
y_train_relation_labels_full.shape

(1095, 30)

In [71]:
y_train_relation_labels_full[1000]

array([98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98,
       98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98], dtype=int32)

In [72]:
y_train_ohe = np.zeros((y_train.shape[0], max_len, 5), dtype='int32')
for row, i in zip(y_train_full[:, :], range(y_train_full.shape[0])):
    y_train_instance_matrix = label_binarizer.transform(row)
    y_train_ohe[i, :, :] = y_train_instance_matrix

In [73]:
max_len

30

In [74]:
y_train_relation_ohe = np.zeros((y_train_relation_labels_full.shape[0], max_len, len(relations)), dtype='int32')
for row, i in zip(y_train_relation_labels_full[:, :], range(y_train_relation_labels_full.shape[0])):
    for j in range(max_len):
        relation = row[j]
        if relation != NO_RELATION_LABEL:
            y_train_relation_ohe[i, j, relation] = 1

In [75]:
y_train_relation_ohe.shape

(1095, 30, 98)

In [76]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Flatten, TimeDistributed
from keras.models import Model

buffer_input = Input(shape=(max_len,), dtype='int32')
stack_input_0 = Input(shape=(max_len,), dtype='int32')
stack_input_1 = Input(shape=(max_len,), dtype='int32')
stack_input_2 = Input(shape=(max_len,), dtype='int32')
prev_action_input = Input(shape=(max_len, 5), dtype='float32')
dep_info_input = Input(shape=(max_len, 6), dtype='float32')

embedding = Embedding(len(word_index) + 2,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

buffer_emb = embedding(buffer_input)
stack_emb_0 = embedding(stack_input_0)
stack_emb_1 = embedding(stack_input_1)
stack_emb_2 = embedding(stack_input_2)

x = concatenate([buffer_emb, stack_emb_0, stack_emb_1, stack_emb_2, prev_action_input, dep_info_input])

lstm_output = LSTM(1024, return_sequences=True)(x)

action = TimeDistributed(Dense(5, activation="softmax"))(lstm_output)
relation = TimeDistributed(Dense(len(relations), activation="softmax"))(lstm_output)

In [77]:
lstm_output

<tf.Tensor 'lstm_1/transpose_1:0' shape=(?, ?, 1024) dtype=float32>

In [78]:
action

<tf.Tensor 'time_distributed_1/Reshape_1:0' shape=(?, 30, 5) dtype=float32>

In [79]:
relation

<tf.Tensor 'time_distributed_2/Reshape_1:0' shape=(?, 30, 98) dtype=float32>

In [80]:
model = Model([buffer_input, stack_input_0, stack_input_1, stack_input_2, prev_action_input, dep_info_input], 
              [action, relation])

In [81]:
from keras.optimizers import RMSprop, Adam
rms = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
model.compile(optimizer=rms,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [82]:
from keras.utils import plot_model

plot_model(model, to_file='model.png')

In [83]:
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 30)            0                                            
___________________________________________________________________________________________

In [84]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

history = model.fit([x_train_full[:, :, 0], x_train_full[:, :, 1], x_train_full[:, :, 2], x_train_full[:, :, 3], x_train_full[:, :, 4:9], x_train_full[:, :, 9:]], 
         [y_train_ohe, y_train_relation_ohe],
         epochs=70, batch_size=16,
         validation_split=0.2,
         callbacks=[ModelCheckpoint('./dfa_model_with_deps_relations', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)])

Train on 876 samples, validate on 219 samples
Epoch 1/70
876/876 [==============================] - 32s - loss: 0.6255 - time_distributed_1_loss: 0.3783 - time_distributed_2_loss: 0.2472 - time_distributed_1_acc: 0.3652 - time_distributed_2_acc: 0.1220 - val_loss: 0.4435 - val_time_distributed_1_loss: 0.2560 - val_time_distributed_2_loss: 0.1875 - val_time_distributed_1_acc: 0.3429 - val_time_distributed_2_acc: 0.1107

In [96]:
import pickle as pickle

f = open('history', 'w')
pickle.dump(history.history, f)

In [85]:
model.load_weights('./models/dfa_model_with_deps', by_name=False)
index_to_word_map = {v: k for k, v in tokenizer.word_index.iteritems()}

def get_predictions_from_distr(predictions_distr):
    predictions = [np.argmax(p) for p in predictions_distr]
    return predictions

def pretty_print_predictions(predictions):
    actions = ['SH', 'RL', 'RR', 'DN', 'SW']
    for i in range(len(predictions)):
        print actions[predictions[i]],;
    print '\n'
    
def pretty_print_sentence(tokens, index_to_word_map):
    for i in range(len(tokens)):
        print index_to_word_map[tokens[i]],;
    print '\n'

def make_prediction(model, x_test, y_test):
    tokens_buffer = x_test
    tokens_stack = []
    current_step = 0
    buffer_token = np.zeros((1, max_len))
    stack_token0 = np.zeros((1, max_len))
    stack_token1 = np.zeros((1, max_len))
    stack_token2 = np.zeros((1, max_len))
    prev_action = np.zeros((1, max_len, 5))
    
    buffer_token[0][current_step] = tokens_buffer[0]
    stack_token0[0][current_step] = no_word_index
    stack_token1[0][current_step] = no_word_index
    stack_token2[0][current_step] = no_word_index
    prev_action[0][current_step] = [0, 0, 0, 0, 0]

    final_prediction = []
    while (len(tokens_buffer) != 0 or len(tokens_stack) != 1) and current_step < max_len - 1:
        prediction = model.predict([buffer_token, stack_token0, stack_token1, stack_token2, prev_action])
        current_actions_distr_ordered = np.argsort(prediction[0][current_step])[::-1]
        current_inspected_action_index = 0
        current_action = current_actions_distr_ordered[current_inspected_action_index]
        invalid = True
        while invalid:
            invalid = False
            current_action = current_actions_distr_ordered[current_inspected_action_index]
            current_inspected_action_index += 1
            if current_action == SH:
                if len(tokens_buffer) == 0:
                    invalid = True
                    continue
                tokens_stack = [tokens_buffer[0]] + tokens_stack
                tokens_buffer = tokens_buffer[1:]
            if  current_action == RL:
                if len(tokens_stack) < 2:
                    invalid = True
                    continue
                tokens_stack = [tokens_stack[0]] + tokens_stack[2:]
            if current_action == RR:
                if len(tokens_stack) < 2:
                    invalid = True
                    continue
                tokens_stack = [tokens_stack[1]] + tokens_stack[2:]
            if current_action == DN:
                if len(tokens_buffer) == 0:
                    invalid = True
                    continue
                tokens_buffer = tokens_buffer[1:]
            if current_action == SW:
                if len(tokens_stack) < 3:
                    invalid = True
                    continue
                tokens_stack = [tokens_stack[0], tokens_stack[2], tokens_stack[1]] + tokens_stack[3:]
        final_prediction.append(current_action)
        current_step += 1
        if len(tokens_buffer) > 0:
            buffer_token[0][current_step] = tokens_buffer[0]
        else:
            buffer_token[0][current_step] = no_word_index
            
        if len(tokens_stack) > 3:
            stack_token0[0][current_step] = tokens_stack[0]
            stack_token1[0][current_step] = tokens_stack[1]
            stack_token2[0][current_step] = tokens_stack[2]
        else:
            if len(tokens_stack) > 2:
                stack_token0[0][current_step] = tokens_stack[0]
                stack_token1[0][current_step] = tokens_stack[1]
                stack_token2[0][current_step] = no_word_index
            else:
                if len(tokens_stack) > 1:
                    stack_token0[0][current_step] = tokens_stack[0]
                    stack_token1[0][current_step] = no_word_index
                    stack_token2[0][current_step] = no_word_index
                else:
                    stack_token0[0][current_step] = no_word_index
                    stack_token1[0][current_step] = no_word_index
                    stack_token2[0][current_step] = no_word_index
        prev_action[0][current_step] = label_binarizer.transform([current_action])[0, :]
    print 'Buffer and stack at end of prediction'
    print tokens_buffer
    print tokens_stack
    return final_prediction

for i in range(10):
    prediction = make_prediction(model, x_test[i], y_test[i])  
    print 'Predicted'
    pretty_print_predictions(prediction)
    print 'Actual'
    pretty_print_predictions(y_test[i])
    print 'Sentence'
    pretty_print_sentence(x_test[i], index_to_word_map)    
    print 'Amr'
    print amrs_test[i]

ValueError: You are trying to load a weight file containing 3 layers into a model with 4 layers.